In [1]:
fn_answers     = '/root/data/answers.json'
# fn_predictions = '/root/results/2023-05-04-12_53_56.txt'

In [2]:
fn_predictions_list = ["/root/results/2023-04-30-21_01_11.txt"  ,
                       "/root/results/2023-05-01-22_30_58.txt"  ,
                       "/root/results/2023-05-02-21_42_04.txt"  ,
                       "/root/results/2023-05-01-06_42_40.txt"  ,
                        "/root/results/2023-05-01-23_20_30.txt"  ,
                        "/root/results/2023-05-03-08_20_57.txt",
                        "/root/results/2023-05-01-22_13_30.txt"  ,
                        "/root/results/2023-05-02-21_17_36.txt"  ,
                        "/root/results/2023-05-04-12_53_56.txt"]

In [3]:
from T2CEvaluator import T2CEvaluator
evaluator = T2CEvaluator()
for fn_predictions in fn_predictions_list:
    print(fn_predictions)
    !python /root/code/CodeXGLUE_t2c_evaluator/evaluator.py -a {fn_answers} -p {fn_predictions}
    print(evaluator.calculate_metrics(fn_answers = fn_answers, 
                                      fn_predictions = fn_predictions)
         )

/root/results/2023-04-30-21_01_11.txt
INFO:__main__:BLEU: 9.56, EM: 0.0
{'EM': 0.0, 'BLEU': 0.09559306017454255, 'brevity_penalty': 0.6567345865799272, 'ratio': 0.703989703989704, 'translation_length': 2188, 'reference_length': 3108, 'precisions_0': 0.3074463225216994, 'precisions_1': 0.19435136428913355, 'precisions_2': 0.11389297987236131, 'precisions_3': 0.06596173212487412}
/root/results/2023-05-01-22_30_58.txt
INFO:__main__:BLEU: 6.82, EM: 0.0
{'EM': 0.0, 'BLEU': 0.06819640430415487, 'brevity_penalty': 0.7251124219221319, 'ratio': 0.7567567567567568, 'translation_length': 2352, 'reference_length': 3108, 'precisions_0': 0.1878453038674033, 'precisions_1': 0.12028406569019086, 'precisions_2': 0.07345651194231635, 'precisions_3': 0.0471395881006865}
/root/results/2023-05-02-21_42_04.txt
INFO:__main__:BLEU: 5.27, EM: 0.0
{'EM': 0.0, 'BLEU': 0.0527028556912851, 'brevity_penalty': 0.48166219569389485, 'ratio': 0.5778635778635779, 'translation_length': 1796, 'reference_length': 3108, 'pr

In [2]:
!ls

T2CEvaluator.py			__pycache__		temp.txt
YT_Alpaca_7B_LoRa_v4.ipynb	bleu_metric_test.ipynb
YT_Alpaca_7B_LoRa_v4_old.ipynb	lora-alpaca


In [3]:
!ls /root/results/

2023-04-30-21_01_11.txt  2023-05-01-22_30_58.txt  2023-05-02-21_42_04.txt
2023-05-01-06_42_40.txt  2023-05-01-23_20_30.txt  2023-05-03-08_20_57.txt
2023-05-01-22_13_30.txt  2023-05-02-21_17_36.txt  2023-05-04-12_53_56.txt


In [ ]:
for fn_predictions in fn_predictions_list:
    print(fn_predictions)
    !python /root/code/CodeXGLUE_t2c_evaluator/evaluator.py -a {fn_answers} -p {fn_predictions}

In [4]:
!python /root/code/CodeXGLUE_t2c_evaluator/evaluator.py -a {fn_answers} -p {fn_predictions}

INFO:__main__:BLEU: 9.41, EM: 0.0


In [3]:
# # Copyright 2017 Google Inc. All Rights Reserved.
# #
# # Licensed under the Apache License, Version 2.0 (the "License");
# # you may not use this file except in compliance with the License.
# # You may obtain a copy of the License at
# #
# #     http://www.apache.org/licenses/LICENSE-2.0
# #
# # Unless required by applicable law or agreed to in writing, software
# # distributed under the License is distributed on an "AS IS" BASIS,
# # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# # See the License for the specific language governing permissions and
# # limitations under the License.
# # ==============================================================================

# """Python implementation of BLEU and smooth-BLEU.
# This module provides a Python implementation of BLEU and smooth-BLEU.
# Smooth BLEU is computed following the method outlined in the paper:
# Chin-Yew Lin, Franz Josef Och. ORANGE: a method for evaluating automatic
# evaluation metrics for machine translation. COLING 2004.
# """

# import collections
# import math
# # import os
# # from bleu import _bleu
# import json
# import os
# from typing import Dict

# class T2CEvaluator:
#     def __init__(self, max_order = 4, smooth = True,  verbose = False):
#         self.verbose = verbose
#         self.max_order = max_order
#         self.smooth = smooth
        
    
#     @classmethod
#     def _preprocess_answers(cls, s):
#         s = s.replace('\n', ' ')

#         while '  ' in s:
#             s = s.replace('  ', ' ')

#         while s and s[-1]==' ':
#             s = s[:-1]

#         return s

#     def calculate_metrics(self, fn_answers, fn_predictions, verbose = True) -> Dict:
#         """
#             return EM score, bleu metrics, and others
#         """
#         res  = {}
#         preds = open(fn_predictions, "r").readlines()
#         gts   = open(fn_answers, "r").readlines()

#         assert len(preds) == len(gts), f"Samples of predictions and answers are not equal, {len(preds)}: {len(gts)}"

#         total = len(gts)
#         EM = 0.0

#         translation_corpus = []
#         reference_corpus = []

#         #read the data in correct format
#         for pred, gt in zip(preds, gts):
#             pred = pred.strip()
#             gt = json.loads(gt)["code"]
#             reference_corpus.append([gt.split(' ')])
#             translation_corpus.append(T2CEvaluator._preprocess_answers(pred).split(' '))
#             if pred.split() == gt.split():
#                 EM += 1

#         res['EM'] = EM

#         # 3-Tuple with the BLEU score, n-gram precisions, geometric mean of n-gram
#         #  precisions and brevity penalty.

#         bleu_score, precisions, bp, ratio, translation_length, reference_length = self.compute_bleu(reference_corpus = reference_corpus, 
#                                                                                          translation_corpus= translation_corpus, 
#                                                                                          max_order = self.max_order,
#                                                                                          smooth = self.smooth
#                                                                                         )
#         res['BLEU'] = bleu_score
#         res['brevity_penalty'] = bp
#         res['ratio'] = ratio
#         res['translation_length'] = translation_length
#         res['reference_length'] = reference_length
#         for i in range(len(precisions)):
#             res[f'precisions_{i}'] = precisions[i]

#         if self.verbose:
#             print(f"INFO:__main__:BLEU: {round(100 * bleu_score,2)}, EM: {round(EM/total*100, 2)}") 

#         return res
    
#     @classmethod
#     def _get_ngrams(cls, segment, max_order):
#         """Extracts all n-grams upto a given maximum order from an input segment.
#         Args:
#           segment: text segment from which n-grams will be extracted.
#           max_order: maximum length in tokens of the n-grams returned by this
#               methods.
#         Returns:
#           The Counter containing all n-grams upto max_order in segment
#           with a count of how many times each n-gram occurred.
#         """
#         ngram_counts = collections.Counter()
#         for order in range(1, max_order + 1):
#             for i in range(0, len(segment) - order + 1):
#                 ngram = tuple(segment[i:i+order])
#                 ngram_counts[ngram] += 1
#         return ngram_counts

#     @classmethod
#     def compute_bleu(cls, reference_corpus, translation_corpus, max_order=4,
#                      smooth=False):
#         """Computes BLEU score of translated segments against one or more references.
#         Args:
#           reference_corpus: list of lists of references for each translation. Each
#               reference should be tokenized into a list of tokens.
#           translation_corpus: list of translations to score. Each translation
#               should be tokenized into a list of tokens.
#           max_order: Maximum n-gram order to use when computing BLEU score.
#           smooth: Whether or not to apply Lin et al. 2004 smoothing.
#         Returns:
#           3-Tuple with the BLEU score, n-gram precisions, geometric mean of n-gram
#           precisions and brevity penalty.
#         """
#         matches_by_order = [0] * max_order
#         possible_matches_by_order = [0] * max_order
#         reference_length = 0
#         translation_length = 0
#         for (references, translation) in zip(reference_corpus,
#                                              translation_corpus):
#             reference_length += min(len(r) for r in references)
#             translation_length += len(translation)

#             merged_ref_ngram_counts = collections.Counter()
#             for reference in references:
#                 merged_ref_ngram_counts |= cls._get_ngrams(reference, max_order)
#             translation_ngram_counts = cls._get_ngrams(translation, max_order)
#             overlap = translation_ngram_counts & merged_ref_ngram_counts
#             for ngram in overlap:
#                 matches_by_order[len(ngram)-1] += overlap[ngram]
#             for order in range(1, max_order+1):
#                 possible_matches = len(translation) - order + 1
#                 if possible_matches > 0:
#                     possible_matches_by_order[order-1] += possible_matches

#         precisions = [0] * max_order
#         for i in range(0, max_order):
#             if smooth:
#                 precisions[i] = ((matches_by_order[i] + 1.) /
#                                  (possible_matches_by_order[i] + 1.))
#             else:
#                 if possible_matches_by_order[i] > 0:
#                     precisions[i] = (float(matches_by_order[i]) /
#                                      possible_matches_by_order[i])
#                 else:
#                     precisions[i] = 0.0

#         if min(precisions) > 0:
#             p_log_sum = sum((1. / max_order) * math.log(p) for p in precisions)
#             geo_mean = math.exp(p_log_sum)
#         else:
#             geo_mean = 0

#         ratio = float(translation_length) / reference_length

#         if ratio > 1.0:
#             bp = 1.
#         else:
#             bp = math.exp(1 - 1. / ratio)

#         bleu = geo_mean * bp

#         return (bleu, precisions, bp, ratio, translation_length, reference_length)

In [29]:
# t2c_evaluator = T2CEvaluator()

# for fn_predictions in fn_predictions_list:
#     print(fn_predictions)
#     !python /root/code/CodeXGLUE_t2c_evaluator/evaluator.py -a {fn_answers} -p {fn_predictions}

#     print(t2c_evaluator.calculate_metrics(fn_answers = fn_answers, 
#                                     fn_predictions = fn_predictions
#                                     )["BLEU"]*100
#          )

In [6]:
# # import os
# # from bleu import _bleu
# import json
# import os

# def _preprocess_answers(s):
#     s = s.replace('\n', ' ')
    
#     while '  ' in s:
#         s = s.replace('  ', ' ')
        
#     while s and s[-1]==' ':
#         s = s[:-1]
    
#     return s

# def calculate_metrics(fn_answers, fn_predictions, verbose = True):
#     """
#         return bleu metrics and EM score
#     """
#     res  = {}
#     preds = open(fn_predictions, "r").readlines()
#     gts   = open(fn_answers, "r").readlines()

#     assert len(preds) == len(gts), f"Samples of predictions and answers are not equal, {len(preds)}: {len(gts)}"

#     total = len(gts)
#     EM = 0.0
    
#     translation_corpus = []
#     reference_corpus = []

#     #read the data in correct format
#     for pred, gt in zip(preds, gts):
#         pred = pred.strip()
#         gt = json.loads(gt)["code"]
#         reference_corpus.append([gt.split(' ')])
#         translation_corpus.append(_preprocess_answers(pred).split(' '))
#         if pred.split() == gt.split():
#             EM += 1
    
#     res['EM'] = EM

#     # 3-Tuple with the BLEU score, n-gram precisions, geometric mean of n-gram
#     #  precisions and brevity penalty.

#     bleu_score, precisions, bp, ratio, translation_length, reference_length = compute_bleu(reference_corpus = reference_corpus, 
#                                                                                      translation_corpus= translation_corpus, 
#                                                                                      max_order = 4, 
#                                                                                      smooth = True
#                                                                                     )
#     res['BLEU'] = bleu_score
#     res['brevity_penalty'] = bp
#     res['ratio'] = ratio
#     res['translation_length'] = translation_length
#     res['reference_length'] = reference_length
#     for i in range(len(precisions)):
#         res[f'precisions_{i}'] = precisions[i]
        
#     if verbose:
#         print(f"INFO:__main__:BLEU: {round(100 * bleu_score,2)}, EM: {round(EM/total*100, 2)}") 
    
#     return res

# calculate_metrics(fn_answers = fn_answers, 
#                   fn_predictions = fn_predictions
#                  )

INFO:__main__:BLEU: 9.41, EM: 0.0


{'EM': 0.0,
 'BLEU': 0.09414554673739628,
 'brevity_penalty': 1.0,
 'ratio': 1.0386100386100385,
 'translation_length': 3228,
 'reference_length': 3108,
 'precisions_0': 0.2003716320842366,
 'precisions_1': 0.12304250559284116,
 'precisions_2': 0.07168342523516055,
 'precisions_3': 0.04445176160684886}

In [7]:
# compute_bleu(reference_corpus = reference_corpus, 
#                                              translation_corpus= translation_corpus, 
#                                              max_order = 4, 
#                                              smooth = True
#                                             )

In [9]:
for fn_predictions in fn_predictions_list:
    print(fn_predictions)
    !python /root/code/CodeXGLUE_t2c_evaluator/evaluator.py -a {fn_answers} -p {fn_predictions}
    print(calculate_metrics(fn_answers = fn_answers, 
                  fn_predictions = fn_predictions
                 )
         )

/root/results/2023-04-30-21_01_11.txt
INFO:__main__:BLEU: 9.56, EM: 0.0
INFO:__main__:BLEU: 9.56, EM: 0.0
{'EM': 0.0, 'BLEU': 0.09559306017454255, 'brevity_penalty': 0.6567345865799272, 'ratio': 0.703989703989704, 'translation_length': 2188, 'reference_length': 3108, 'precisions_0': 0.3074463225216994, 'precisions_1': 0.19435136428913355, 'precisions_2': 0.11389297987236131, 'precisions_3': 0.06596173212487412}
/root/results/2023-05-01-22_30_58.txt
INFO:__main__:BLEU: 6.82, EM: 0.0
INFO:__main__:BLEU: 6.82, EM: 0.0
{'EM': 0.0, 'BLEU': 0.06819640430415487, 'brevity_penalty': 0.7251124219221319, 'ratio': 0.7567567567567568, 'translation_length': 2352, 'reference_length': 3108, 'precisions_0': 0.1878453038674033, 'precisions_1': 0.12028406569019086, 'precisions_2': 0.07345651194231635, 'precisions_3': 0.0471395881006865}
/root/results/2023-05-02-21_42_04.txt
INFO:__main__:BLEU: 5.27, EM: 0.0
INFO:__main__:BLEU: 5.27, EM: 0.0
{'EM': 0.0, 'BLEU': 0.0527028556912851, 'brevity_penalty': 0.481

In [155]:
gt_list[0]

'int function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }'

In [158]:
reference_text[0][0]

'int function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }\n'

['int',
 'function',
 '(',
 ')',
 '{',
 '_total',
 '=',
 'extractList',
 '(',
 ')',
 '.',
 'size',
 '(',
 ')',
 ';',
 'return',
 '_total',
 ';',
 '}']

In [167]:
# reference_text
# gt_list
# reference_text


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [121]:
preds[-14].replace('\n', ''),' '.join(translations[-14])

('void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll ',
 'void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll void removeAll')

In [128]:
per_segment_references = []
for references in zip(*reference_text):
    reference_list = []
    for reference in references:
        reference_list.append(reference.strip().split())
    per_segment_references.append(reference_list)

def simple_preprocess(s):
    s = s.replace('\n', ' ')
    
    while '  ' in s:
        s = s.replace('  ', ' ')
        
    while s and s[-1]==' ':
        s = s[:-1]
    
    return s

compute_bleu(reference_corpus = per_segment_references, 
             translation_corpus = translations,
             max_order = 4, 
             smooth = True)

(0.09414554673739628,
 [0.2003716320842366,
  0.12304250559284116,
  0.07168342523516055,
  0.04445176160684886],
 1.0,
 1.0386100386100385,
 3228,
 3108)

In [122]:
# [p.replace('\n', '').split(' ')==y for p, y in zip(preds, translations)][:-13]

In [129]:
reference_text[0]

['int function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }\n',
 'void function ( boolean arg0 ) { fStatementsRecovery = arg0 ; }\n',
 'boolean function ( ) { return false ; }\n',
 'String function ( String arg0 ) { return ZKUtil . joinZNode ( getIdentitiesZNode ( ) , arg0 ) ; }\n',
 'int function ( GenVector < C > arg0 ) { if ( ! modul . equals ( arg0 . modul ) ) { return - 1 ; } List < C > loc0 = arg0 . val ; int loc1 = 0 ; for ( C loc2 : val ) { int loc3 = loc2 . compareTo ( loc0 . get ( loc1 ++ ) ) ; if ( loc3 != 0 ) { return loc3 ; } } return 0 ; }\n',
 'void function ( int arg0 , int arg1 ) { fTagPosition = new Region ( arg0 , arg1 ) ; }\n',
 'double function ( UnivariateRealFunction arg0 , double arg1 , double arg2 ) { setup ( arg0 ) ; return factory . newDefaultSolver ( arg0 ) . solve ( arg1 , arg2 ) ; }\n',
 'boolean function ( ) { return Platform . isWindows ( ) || Platform . isOSX ( ) || Platform . JAVA_VERSION < Platform . JAVA_1_3 ; }\n',
 'Integer functio

In [ ]:
any

In [127]:
per_segment_references = []
for references in zip(*reference_text):
    reference_list = []
    for reference in references:
        reference_list.append(reference.strip().split())
    per_segment_references.append(reference_list)

def simple_preprocess(s):
    s = s.replace('\n', ' ')
    
    while '  ' in s:
        s = s.replace('  ', ' ')
        
    while s and s[-1]==' ':
        s = s[:-1]
    
    return s

compute_bleu(reference_corpus = per_segment_references, 
             translation_corpus = [simple_preprocess(p).split(' ') for p in preds], 
             max_order = 4, 
             smooth = True)

(0.09414554673739628,
 [0.2003716320842366,
  0.12304250559284116,
  0.07168342523516055,
  0.04445176160684886],
 1.0,
 1.0386100386100385,
 3228,
 3108)

In [80]:
# preds[1], gts[1]

('if ( fStatementsRecovery ) { fRewrite = fRewrite . getRewrite ( ) ; fImportRewrite = fImportRewrite . getRewrite ( ) ; fImportRemover = fImportRemover . getRewrite ( ) ; fResolveBindings = fResolveBindings . getRewrite ( ) ; } } return fStatementsRecovery ; } return false ; } } return fRewrite . getRewrite ( ) . getStatementsRecovery ( ) ; } } return false ; } } return fRewrite . getRewrite ( ) . getStatements\n',
 '{"code": "void function ( boolean arg0 ) { fStatementsRecovery = arg0 ; }", "nl": "controls whether the compiler should perform statements recovery . to be effective , this method must be called before any of #getroot , #getastrewrite , #getimportremover . this method has no effect if the target object has been created with #compilationunitrewrite icompilationunit , compilationunit . defaults to true do perform statements recovery . concode_field_sep ImportRewrite fImportRewrite concode_elem_sep ImportRemover fImportRemover concode_elem_sep WorkingCopyOwner fOwner concode

In [67]:
json.loads(gts[0])["code"]

'int function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }'

In [69]:
[i[:-1] for i in reference_text[0]]

['int function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }',
 'void function ( boolean arg0 ) { fStatementsRecovery = arg0 ; }',
 'boolean function ( ) { return false ; }',
 'String function ( String arg0 ) { return ZKUtil . joinZNode ( getIdentitiesZNode ( ) , arg0 ) ; }',
 'int function ( GenVector < C > arg0 ) { if ( ! modul . equals ( arg0 . modul ) ) { return - 1 ; } List < C > loc0 = arg0 . val ; int loc1 = 0 ; for ( C loc2 : val ) { int loc3 = loc2 . compareTo ( loc0 . get ( loc1 ++ ) ) ; if ( loc3 != 0 ) { return loc3 ; } } return 0 ; }',
 'void function ( int arg0 , int arg1 ) { fTagPosition = new Region ( arg0 , arg1 ) ; }',
 'double function ( UnivariateRealFunction arg0 , double arg1 , double arg2 ) { setup ( arg0 ) ; return factory . newDefaultSolver ( arg0 ) . solve ( arg1 , arg2 ) ; }',
 'boolean function ( ) { return Platform . isWindows ( ) || Platform . isOSX ( ) || Platform . JAVA_VERSION < Platform . JAVA_1_3 ; }',
 'Integer function ( ) { return n

In [61]:
# from nltk.translate.bleu_score import sentence_bleu
# from torchmetrics import BLEUScore
# preds = ['the cat is on the mat']
# target = [['there is a cat on the mat', 'a cat is on the mat']]
# bleu = BLEUScore()
# bleu(preds, target)

['int function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }',
 'void function ( boolean arg0 ) { fStatementsRecovery = arg0 ; }',
 'boolean function ( ) { return false ; }',
 'String function ( String arg0 ) { return ZKUtil . joinZNode ( getIdentitiesZNode ( ) , arg0 ) ; }',
 'int function ( GenVector < C > arg0 ) { if ( ! modul . equals ( arg0 . modul ) ) { return - 1 ; } List < C > loc0 = arg0 . val ; int loc1 = 0 ; for ( C loc2 : val ) { int loc3 = loc2 . compareTo ( loc0 . get ( loc1 ++ ) ) ; if ( loc3 != 0 ) { return loc3 ; } } return 0 ; }',
 'void function ( int arg0 , int arg1 ) { fTagPosition = new Region ( arg0 , arg1 ) ; }',
 'double function ( UnivariateRealFunction arg0 , double arg1 , double arg2 ) { setup ( arg0 ) ; return factory . newDefaultSolver ( arg0 ) . solve ( arg1 , arg2 ) ; }',
 'boolean function ( ) { return Platform . isWindows ( ) || Platform . isOSX ( ) || Platform . JAVA_VERSION < Platform . JAVA_1_3 ; }',
 'Integer function ( ) { return n

In [51]:
preds[0]

'-\n'

In [56]:
json.loads(gts[0])["code"]

'int function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }'

In [57]:
reference_text[0][0]#, translations 

'int function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }\n'

In [58]:
gts[1]

'{"code": "void function ( boolean arg0 ) { fStatementsRecovery = arg0 ; }", "nl": "controls whether the compiler should perform statements recovery . to be effective , this method must be called before any of #getroot , #getastrewrite , #getimportremover . this method has no effect if the target object has been created with #compilationunitrewrite icompilationunit , compilationunit . defaults to true do perform statements recovery . concode_field_sep ImportRewrite fImportRewrite concode_elem_sep ImportRemover fImportRemover concode_elem_sep WorkingCopyOwner fOwner concode_elem_sep ASTRewrite fRewrite concode_elem_sep boolean fStatementsRecovery concode_elem_sep List<TextEditGroup> fTextEditGroups concode_elem_sep ICompilationUnit fCu concode_elem_sep IDocument fRememberContent concode_elem_sep CompilationUnit fRoot concode_elem_sep boolean fResolveBindings concode_elem_sep boolean fBindingsRecovery concode_field_sep void clearASTRewrite concode_elem_sep ASTRewrite getASTRewrite concod